# EXAMEN 2

WelfareBnk es una fintech que incursionó hace cinco años en el mercado Mexicano (adquirió una
instituci´on financiera que ten´ıa cincuenta años en el pa´ıs). Actualmente, la institución tiene diversos
problemas relacionados con la cartera vencida y la pérdida de clientes, por lo que ha solicitado tu empresa
de consultoría el desarrollo de dos servicios/aplicaciones que le ayuden a:

Identificar clientes que potencialmente abandonar´an el banco. Esto les permitirá anticiparse y ofrecer
a los clientes incentivos para no abandonar la institución.
Noviembre 2024 ITESO, Universidad Jesuita de Guadalajara
Reducir la cartera vencida, mediante una mejor evaluación de riesgos sobre la solicitudes de créditos
realizadas por los clientes.

Como ingeniero con experiencia en aprendizaje automático se te ha asignado la construcción, entre-
namiento y validación de modelos que puedan ser utilizados como parte de los servicios que ayudarán
a la empresa a evaluar mejor los riesgos de cr´edito al consumidor y la identificaci´on de clientes que
potencialmente abandonarán la institución.

In [95]:
# Firstly we must unzip the songs data and metadata
'''
import zipfile
with zipfile.ZipFile('./muestra_datos_WelfareBnk.zip', 'r') as zip_ref:
    zip_ref.extractall('./data')
'''

"\nimport zipfile\nwith zipfile.ZipFile('./muestra_datos_WelfareBnk.zip', 'r') as zip_ref:\n    zip_ref.extractall('./data')\n"

In [96]:
import numpy as np
import pandas as pd
import tensorflow as tf

#Load the CSV file into a Pandas DataFrame
risk = './data/riesgo_crediticio.csv'
abandonment = './data/abandono_de_clientes.csv'
risk_d = pd.read_csv(risk, delimiter=',')
abandonment_d = pd.read_csv(abandonment, delimiter=',')
#print(risk_d.head())
#print(abandonment_d.head())

# Print the column names to debug
#print("Column names:", risk_d.columns)
#print("Column names:", abandonment_d.columns)

# Extract target column for each dataset
risk_Y = risk_d.pop('grado_credito')
abandonment_Y = abandonment_d.pop('abandono')

print(risk_Y.value_counts())

# Extract the features for each dataset
# For Risk, extract Historial crediticio, Incumplimiento histórico, Monto de crédito, Proporción de ingreso and Antigüedad en el empleo
risk_X = risk_d[['historial_crediticio', 'incumplimiento_historico', 'monto_credito', 'procentaje_de_ingreso', 'antiguedad_empleo', 'ingreso_anual', 'monto_credito', 'tasa_interes']]
risk_X['incumplimiento_historico'] = risk_X['incumplimiento_historico'].replace(['Y', 'N'], [1.0, 0.0])

# Since the target risk_y has a low count for some classes, we will create 3 classifications they can get
# 0 - A, B, 1 - C, 2 - D, E, F, G, representing low, medium and high risk respectively
risk_Y = risk_Y.replace(['A', 'B'], 0)
risk_Y = risk_Y.replace(['C'], 1)
risk_Y = risk_Y.replace(['D', 'E', 'F', 'G'], 2)

# Balance the target variable for the Risk dataset
from imblearn.under_sampling import ClusterCentroids  

# Check for balance in the target variable
print(risk_Y.value_counts())

for column in risk_X.columns:
    risk_X[column] = risk_X[column].fillna(0)

clustercentroids = ClusterCentroids()
risk_X, risk_Y = clustercentroids.fit_resample(risk_X, risk_Y)

# Check for balance in the target variable
print(risk_Y.value_counts())

# Scale the features for the Risk dataset
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
risk_X = scaler.fit_transform(risk_X)

# For Abandonment, extract Permanencia, Es cliente activo, Saldo, Puntuación crediticia
abandonment_X = abandonment_d[['permanencia', 'es_cliente_activo', 'saldo', 'puntuacion_crediticia']]

# Check the shape of the data
print(risk_X.shape)
print(risk_Y.shape)

print(abandonment_X.shape)
print(abandonment_Y.shape)

# Check for balance in the target variable
#print(risk_Y.value_counts())
#print(abandonment_Y.value_counts())

# Count the number of missing values
#print(risk_X.isnull().sum())
#print(abandonment_X.isnull().sum())

# Count the number of missing values
# print(risk_X.isnull().sum())
# print(abandonment_X.isnull().sum())

# Balance the target variable for the abandonment dataset

clustercentroids = ClusterCentroids()
abandonment_X, abandonment_Y = clustercentroids.fit_resample(abandonment_X, abandonment_Y)

# Check the shape of the data
print(risk_X.shape)
print(risk_Y.shape)

print(abandonment_X.shape)
print(abandonment_Y.shape)

# Check for balance in the target variable
# print(risk_Y.value_counts())
# print(abandonment_Y.value_counts())

grado_credito
A    10777
B    10451
C     6458
D     3626
E      964
F      241
G       64
Name: count, dtype: int64
grado_credito
0    21228
1     6458
2     4895
Name: count, dtype: int64


C:\Users\acasi\AppData\Local\Temp\ipykernel_21236\1063641756.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  risk_X['incumplimiento_historico'] = risk_X['incumplimiento_historico'].replace(['Y', 'N'], [1.0, 0.0])
C:\Users\acasi\AppData\Local\Temp\ipykernel_21236\1063641756.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  risk_X['incumplimiento_historico'] = risk_X['incumplimiento_historico'].replace(['Y', 'N'], [1.0, 0.0])
C:\Users\acasi\AppData\Local\Temp\ipykernel_21236\1063641756.py:32: FutureWarning: Downc

grado_credito
0    4895
1    4895
2    4895
Name: count, dtype: int64
(14685, 8)
(14685,)
(9960, 4)
(9960,)
(14685, 8)
(14685,)
(4058, 4)
(4058,)


In [97]:
#Split the data into training and testing sets
from sklearn.model_selection import train_test_split

# For Risk
risk_X_train, risk_X_test, risk_Y_train, risk_Y_test = train_test_split(risk_X, risk_Y, test_size=0.2, random_state=42)

# For Abandonment
abandonment_X_train, abandonment_X_test, abandonment_Y_train, abandonment_Y_test = train_test_split(abandonment_X, abandonment_Y, test_size=0.2, random_state=42)

# Convert the DataFrame and the target column to TensorFlow tensors

# For Risk
risk_X_train = tf.constant(risk_X_train, dtype=tf.float32)
risk_X_test = tf.constant(risk_X_test, dtype=tf.float32)
risk_Y_train = tf.constant(risk_Y_train, dtype=tf.float32)
risk_Y_test = tf.constant(risk_Y_test, dtype=tf.float32)

# For Abandonment
abandonment_X_train = tf.constant(abandonment_X_train, dtype=tf.float32)
abandonment_X_test = tf.constant(abandonment_X_test, dtype=tf.float32)
abandonment_Y_train = tf.constant(abandonment_Y_train, dtype=tf.float32)
abandonment_Y_test = tf.constant(abandonment_Y_test, dtype=tf.float32)

# Verify the tensors
print(risk_X_train.shape, risk_X_test.shape, risk_Y_train.shape, risk_Y_test.shape) 
print(abandonment_X_train.shape, abandonment_X_test.shape, abandonment_Y_train.shape, abandonment_Y_test.shape)

(11748, 8) (2937, 8) (11748,) (2937,)
(3246, 4) (812, 4) (3246,) (812,)


In [ ]:
# For the risk model, I will use a radnom forest classifier
from sklearn.ensemble import RandomForestClassifier

# For Risk
risk_model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=None)

# Now we create a Nerual Network model, one for the abandonment dataset, where we will 
# use binary classification for this datset

# For Abandonment
abandonment_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

abandonment_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'Precision'])



In [99]:
# Train the models

# For Risk
risk_model.fit(risk_X_train, risk_Y_train)


# For Abandonment
abandonment_model.fit(abandonment_X_train, abandonment_Y_train, epochs=125)

Epoch 1/125
102/102 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - Precision: 0.6736 - accuracy: 0.5802 - loss: 0.6535
Epoch 2/125
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - Precision: 0.8533 - accuracy: 0.6029 - loss: 0.6240
Epoch 3/125
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - Precision: 0.9772 - accuracy: 0.6281 - loss: 0.6067
Epoch 4/125
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - Precision: 0.9622 - accuracy: 0.6224 - loss: 0.6160
Epoch 5/125
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - Precision: 0.9630 - accuracy: 0.6168 - loss: 0.6132
Epoch 6/125
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - Precision: 0.9625 - accuracy: 0.6062 - loss: 0.6180
Epoch 7/125
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - Precision: 0.9774 - accuracy: 0.6221 - loss: 0.6085
Epoch 8/125
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - Precision: 0.9675 - accuracy: 0.6161 - loss: 0.6114
Epoch 9/125
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - Precision: 0.9651 - accuracy: 0.6154 - loss: 0.6142
Epoch 10/125
102/102 ━━━━━━━━━━━━━━━━

In [100]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
# Test the models

# For Risk

risk_Y_pred = risk_model.predict(risk_X_test)
# print("Risk model prediction:", risk_Y_pred)

print(risk_Y_pred)
print(risk_Y_test.shape)

# Print stats for Risk model predictions

# Round the predictions to 0, 1, or 2
risk_Y_pred = np.round(risk_Y_pred)

# Confusion matrix for the Risk model
cm = confusion_matrix(risk_Y_test, risk_Y_pred)
print("Confusion matrix:")
print(cm)

# Convert tensors to numpy arrays for metric calculation

y_test_np = risk_Y_test.numpy()

# Calculate accuracy, precision, and recall
accuracy = accuracy_score(y_test_np, risk_Y_pred)
precision = precision_score(y_test_np, risk_Y_pred, average='weighted')
recall = recall_score(y_test_np, risk_Y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

# For Abandonment

abandonment_Y_pred = abandonment_model.predict(abandonment_X_test)
# print("Abandonment model predictions:", abandonment_Y_pred)
# print("Abandonment Actual:", abandonment_Y_test)
# Round the predictions to 0 or 1
abandonment_Y_pred = np.round(abandonment_Y_pred)

# Confusion matrix for the Abandonment model
cm = confusion_matrix(abandonment_Y_test, abandonment_Y_pred)
print("Confusion matrix:")
print(cm)

# Convert tensors to numpy arrays for metric calculation
y_test_np = abandonment_Y_test.numpy()

# Calculate accuracy, precision, and recall
accuracy = accuracy_score(y_test_np, abandonment_Y_pred)
precision = precision_score(y_test_np, abandonment_Y_pred)
recall = recall_score(y_test_np, abandonment_Y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")


[2. 1. 2. ... 1. 1. 0.]
(2937,)
Confusion matrix:
[[975  23   7]
 [ 30 850  93]
 [  9 117 833]]
Accuracy: 0.9050051072522982
Precision: 0.9049926743994121
Recall: 0.9050051072522982
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Confusion matrix:
[[429   3]
 [295  85]]
Accuracy: 0.6330049261083743
Precision: 0.9659090909090909
Recall: 0.2236842105263158


In [101]:
# Now we save the models
# python -m pip install skops
import pickle

pickle.dump(risk_model, open('./models/risk_model.pkl', 'wb'))	

abandonment_model.save('./models/abandonment_model.keras', overwrite=True) 